# Spam Detector

*Dataset from https://www.kaggle.com/karthickveerakumar/spam-filter/data * 

In [2]:
# -*- coding: utf-8 -*-
import numpy as np
import pandas as pd

import re

from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.feature_extraction.text import CountVectorizer

import keras
from keras.models import Sequential
from keras.layers import Dense,Dropout

INITIAL = None

Using TensorFlow backend.


In [3]:
dataset = pd.read_csv('emails.csv')
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
import nltk

def getCorpus(data):
    ps = PorterStemmer()
    corpus = []
    for review in data:
        x = re.sub('[^a-zA-Z]',' ',review).lower().split()
        corpus.append(' '.join([ps.stem(word) for word in x if not word in set(stopwords.words('english'))]))
    return corpus

def getWordBag(data):    
    corpus = getCorpus(data)
    maxF = 2000
    cv = CountVectorizer(max_features=maxF)
    X = cv.fit_transform(corpus).toarray()
    
    global INITIAL
    if np.all(INITIAL ==None):
         INITIAL = corpus
            
    if X.shape[1] < maxF:
        y=cv.fit_transform(INITIAL).toarray()
        aux  = [[0]*y.shape[1]]
        w = cv.get_feature_names()
        for word in corpus:
            for y in word.split():
                if y in w:
                    pos = w.index(y)
                    aux[0][pos] = 1
        X = np.array(aux)
    return X

def liked(review):
    if review == [False]:
        return "No es spam."
    else:
        return "Es Spam, Tener cuidado."

In [4]:
from time import time
        
t0 = time()
X = getWordBag(dataset['text'])
t1 = time()
print("el tiempo fue: {} minutos".format((t1-t0)/60))
y = dataset['spam']
X_train , X_test, y_train, y_test = train_test_split(X,y,test_size= 0.25,random_state=0)

el tiempo fue: 10.887896386782328 minutos


In [5]:
Ann = Sequential()
shape = int(X.shape[1]/2)
Ann.add(Dense(units=shape,kernel_initializer='uniform',activation='relu',input_dim=X.shape[1]))
Ann.add(Dropout(0.7))
Ann.add(Dense(units=shape,kernel_initializer='uniform',activation='relu'))
Ann.add(Dropout(0.7))
Ann.add(Dense(units=1,kernel_initializer='uniform',activation='sigmoid'))
        
Ann.compile(optimizer='RMSprop',loss='binary_crossentropy',metrics=['accuracy'])
        

Ann.fit(X_train,y_train,batch_size=10,epochs=10)


Epoch 1/10
4296/4296 [==============================] - 32s 7ms/step - loss: 0.1589 - acc: 0.9574
Epoch 2/10
4296/4296 [==============================] - 30s 7ms/step - loss: 0.0558 - acc: 0.9907
Epoch 3/10
4296/4296 [==============================] - 30s 7ms/step - loss: 0.0587 - acc: 0.9935
Epoch 4/10
4296/4296 [==============================] - 30s 7ms/step - loss: 0.0397 - acc: 0.9953
Epoch 5/10
4296/4296 [==============================] - 30s 7ms/step - loss: 0.0397 - acc: 0.9963
Epoch 6/10
4296/4296 [==============================] - 30s 7ms/step - loss: 0.0366 - acc: 0.9967
Epoch 7/10
4296/4296 [==============================] - 29s 7ms/step - loss: 0.0179 - acc: 0.9981
Epoch 8/10
4296/4296 [==============================] - 30s 7ms/step - loss: 0.0240 - acc: 0.9972
Epoch 9/10
4296/4296 [==============================] - 30s 7ms/step - loss: 0.0194 - acc: 0.9984
Epoch 10/10
4296/4296 [==============================] - 30s 7ms/step - loss: 0.0156 - acc: 0.9988


In [6]:
y_pred = Ann.predict(X_test)
y_pred = (y_pred>0.5)

cm = confusion_matrix(y_test,y_pred)
print(cm)
accuracy = ((cm[0][0]+cm[1][1])/X_test.shape[0])*100
print(accuracy)

[[1081    8]
 [  10  333]]
98.74301675977654


In [8]:
pred = '''Subject: 10 minutes before sex , lasts for 24 - 36 hours  legal , 
            prescription medications under the essential guidance of licensed medical  under every stone lurks a politician .  
            experience is the name everyone gives to their mistakes .  without music , life would be a mistake ."'''

X1 = getWordBag([pred])
y1 = Ann.predict(X1) > 0.5
print(liked(y1))


pred = '''Subject: presentation for cal berkeley  hello vince and john ,  i wanted to forward to you both the current presentations for campus . we can  tweak these however we feel appropriate to match cal berkeley on monday . 
        i  believe that we can probably expect about 30 - 50 students ( based on interest  shown at the career fair ) . these tend to be fairly informal . 
        i was thinking  that we could present in this order :  vince gives the enron overview presentation ( 30 minutes )  john gives the global technology specific presentation ( 20 minutes )  ashley goes over recruiting information at the end ( 10 minutes )  please take a look at each presentation and speaker notes to ensure that you  feel comfortable with the layout and content . 
        i am meeting with john today  at 1 : 30 - vince if you would like to get together and discuss as well that  would be great .  
        if you have any questions , please don ' t hesitate to contact me . 3 - 3589  thanks ,  ashley  vince , here is a copy of the current enron overview presentation . 
        there are  also speaker notes that go into great detail .  john , here is a copy of the current technology presentation for carnegie  mellon . the only changes will be to the recruiting dates at the end . 
        there  are also speaker notes that go into greater detail . - - - >'''
X1 = getWordBag([pred])
y1 = Ann.predict(X1) > 0.5
print(liked(y1))

Es Spam, Tener cuidado.
No es spam.
